In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics


data = pd.read_csv('Sales.csv')
data = pd.get_dummies(data, columns=['Gender'])
data.head()
data.describe()
data.Purchased.value_counts(normalize=True)
data.isnull().sum(axis=0)
X = np.array(data[['User ID','Gender_Female','Gender_Male','Age','EstimatedSalary']])
y = np.array(data[['Purchased']])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.15, 
            random_state = 25, stratify = y)


#method 
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train,y_train)
y_pred = nb.predict(X_test)

print('Accuracy(Multinomial):', metrics.accuracy_score(y_test, y_pred)) 
test_pred = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1,1),y_pred.reshape(-1,1))),
                        columns=['User ID','Gender_Female','Gender_Male','Age','EstimatedSalary',
                             'y_test','y_pred'])
test_pred.head()
test_pred[['y_test','y_pred']].head()



Accuracy(Multinomial): 0.6666666666666666


C:\Users\shehr\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,y_test,y_pred
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0


In [4]:
n=test_pred.shape[0]
n
(sum(test_pred.y_test == test_pred.y_pred)/n)*100
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score, roc_curve,auc
print("accuracy Score=",accuracy_score(y_test,y_pred)*100)

cm = confusion_matrix(y_test,y_pred)
cm


accuracy Score= 66.66666666666666


array([[28, 11],
       [ 9, 12]], dtype=int64)

In [ ]:
TN = cm[0,0]
print("True Negative:",TN)
FP=cm[0,1]
print("False Possitive:",FP)
FN = cm[1,0]
print("False Negative:",FN)
TP = cm[1,1]
print("True Possitive:",TP)
#Again Accuracy
(TN+TP)/n
#TPR(True possitive rate) or Sensitivity
recall = TP/(TP+FP)
recall
#TNR/Specificity
specificity = TN/(TN+FP)
specificity
#FPR
FPR = 1-specificity
FPR
precision = TP/(TP+FP)
precision
#F1_score is the hermonic mean between precision and reall
F1_score = (2*precision*recall)/(precision+recall)
F1_score
#or
f1_score(y_test,y_pred)

## ROC(Receiver operating characteristic) Curve 
#[plot between false positive rate and true positive rate]
probs = nb.predict_proba(X_test)
probs
prob_positive = probs[:, 1]
fpr, tpr, threshold = roc_curve(y_test, prob_positive)

# Reshape each array individually
threshold_reshaped = threshold.reshape(-1, 1)
fpr_reshaped = fpr.reshape(-1, 1)
tpr_reshaped = tpr.reshape(-1, 1)

# Horizontally stack the reshaped arrays
stacked_data = np.hstack((threshold_reshaped, fpr_reshaped, tpr_reshaped))

# Create a DataFrame with appropriate column names
df = pd.DataFrame(stacked_data, columns=['threshold', 'fpr', 'tpr'])

df.head()
roc_auc = auc(fpr,tpr)
roc_auc
import matplotlib.pyplot as plt
plt.title("receicer operating charecteristic")
plt.plot(fpr,tpr,'red',label = 'AUC-%0.2f'% roc_auc)
plt.legend(loc = 'lower right')

plt.plot([0,1],[0,1], color = 'darkblue',linestyle='--')
plt.ylabel('True positive rate')
plt.xlabel('False positive rate')
plt.show()